In [2]:
# %load_ext autoreload
%autoreload
import os, json
from types import SimpleNamespace
from experiment import run_model
from eval import calculate_stats
import pickle
from datetime import datetime 
import torch

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"

In [4]:
config_folder = "config/"

class obj(object):
    def __init__(self, d):
        for a, b in d.items():
            if isinstance(b, (list, tuple)):
               setattr(self, a, [obj(x) if isinstance(x, dict) else x for x in b])
            else:
               setattr(self, a, obj(b) if isinstance(b, dict) else b)

with open(os.path.join(config_folder, "config.json")) as f:
    config = obj(json.load(f))

In [5]:
train_label_ids, train_preds, train_loss, all_dev_label_ids, dev_loss, dev_preds = run_model(config, device)

INFO:root:***** Running training *****
INFO:root:  Num examples = 61014
INFO:root:  Batch size = 12
INFO:root:  Num steps = 50840
Epoch:   0%|                                                                                    | 0/10 [00:00<?, ?it/s]

C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epoch:  10%|███████▎                                                                 | 1/10 [10:32<1:34:55, 632.88s/it]


so far epoch accuracy:  0.9404398990395647


Epoch:  20%|██████████████▌                                                          | 2/10 [21:01<1:24:13, 631.70s/it]


so far epoch accuracy:  0.97618579342446


...

Epoch:  90%|█████████████████████████████████████████████████████████████████▋       | 9/10 [1:34:09<10:27, 627.31s/it]


so far epoch accuracy:  0.9869046448356115


Epoch: 100%|████████████████████████████████████████████████████████████████████████| 10/10 [1:44:35<00:00, 627.54s/it]


so far epoch accuracy:  0.9872324384567477
Training complete
Best val Acc: 0.987232


INFO:root:***** Running training *****
INFO:root:  Num examples = 15304
INFO:root:  Batch size = 12
INFO:root:  Num steps = 12750


In [6]:
train_label_ids, train_preds, train_loss, all_dev_label_ids, dev_loss, dev_preds

(array([0, 8, 5, ..., 6, 1, 8], dtype=int64),
 array([0, 8, 5, ..., 6, 1, 8], dtype=int64),
 0.028281437373610104,
 array([1, 6, 0, ..., 8, 6, 3], dtype=int64),
 0.10090719395104036,
 array([1, 6, 0, ..., 8, 6, 3], dtype=int64))

In [7]:
import numpy as np
np.array(all_dev_label_ids).shape, np.array(dev_preds).shape, np.array(train_preds).shape, np.array(train_label_ids).shape

((15304,), (15304,), (61014,), (61014,))

In [8]:
%autoreload
train_mcc, train_f1_score, train_df_results, train_label_matches_df = calculate_stats(train_label_ids,train_preds )
dev_mcc, dev_f1_score, dev_df_results, dev_label_matches_df = calculate_stats(all_dev_label_ids,dev_preds )


 label: [0 8 5 ... 6 1 8]

 preds: [0 8 5 ... 6 1 8]

 label: [1 6 0 ... 8 6 3]

 preds: [1 6 0 ... 8 6 3]


In [9]:
all_experiment_results = []
all_experiment_results.append([str(config), train_loss, dev_loss, train_mcc, train_f1_score,dev_mcc,dev_f1_score, 
                               all_dev_label_ids, dev_preds,train_label_ids,train_preds  ])
all_experiment_results

[['<__main__.obj object at 0x00000145C2150790>',
  0.028281437373610104,
  0.10090719395104036,
  0.9828714870380854,
  0.9811421345806985,
  0.9655254255851637,
  0.9513203962222588]]

In [10]:
dev_label_matches_df

predicted
labels         matched           
ADE-Drug       False           34
               True           216
Dosage-Drug    False           26
               True           907
Duration-Drug  False            7
               True           129
Form-Drug      False           57
               True           897
Frequency-Drug False           59
               True          1210
Reason-Drug    False           43
               True           991
Route-Drug     False           35
               True          1110
Strength-Drug  False           57
               True          1325
no relation    False           41
               True          8160

##  Try with different configuration

### Just change model from BIOR to BERT Sequence

In [11]:
config.programsettings.MODEL_NAME = "BERT_Sequence"
config.programsettings.DEBUG_PRINT = 0

In [ ]:
train_label_ids, train_preds, train_loss, all_dev_label_ids, dev_loss, dev_preds = run_model(config, device)

INFO:root:***** Running training *****
INFO:root:  Num examples = 61014
INFO:root:  Batch size = 12
INFO:root:  Num steps = 50840
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz from cache at cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.modeling:extracting archive file cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c to temp dir C:\Users\pnarsina\AppData\Local\Temp\2\tmp8umad1wl
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_v

C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


In [ ]:

train_mcc, train_f1_score, train_df_results, train_label_matches_df = calculate_stats(train_label_ids,train_preds )
dev_mcc, dev_f1_score, dev_df_results, dev_label_matches_df = calculate_stats(all_dev_label_ids,dev_preds)

In [ ]:
all_experiment_results.append([str(config), train_loss, dev_loss, train_mcc, train_f1_score,dev_mcc,dev_f1_score, 
                               all_dev_label_ids, dev_preds,train_label_ids,train_preds  ])
all_experiment_results

In [ ]:
dev_label_matches_df

In [ ]:

all_model_results_pickle_file = config.programsettings.REPORTS_DIR + "multi_model_experiment_results_" + str(datetime.now()).replace(":", "_").replace(".", "_") + ".pkl"
with open(all_model_results_pickle_file, "wb") as f:
    pickle.dump(all_experiment_results, f)  


In [ ]:
for i in range(0,100):
    print (all_dev_label_ids.numpy()[i], dev_preds[i])
